# AI 모델 구현: 온도 및 전령량 예측

온도 및 사용된 전령량을 예측하고 각 센서의 예측값을 REST API로 통해 반환할 수 있도록 구현하였습니다.

---

## 1. Import

In [ ]:
from influxdb_client import InfluxDBClient
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import pmdarima as pm
from tensorflow.keras.models import Sequential  # type: ignore
from tensorflow.keras.layers import LSTM, Dense  # type: ignore
from dotenv import load_dotenv
import os

## 2. 데이터 수집

- ### 환경 변수 적용하기

In [ ]:
# InfluxDB 접속 정보 불러오기
load_dotenv('parameters.env')

DB_URL = os.getenv('DB_URL')
DB_TOKEN = os.getenv('DB_TOKEN')
DB_ORG = os.getenv('DB_ORG')
DB_BUCKET = os.getenv('DB_BUCKET')

- ### InfluxDB에 접속하여 필요한 센서 데이터 가져오기

In [ ]:
# 온도 데이터 가져오기
def query_sensor_data(self, branch):
    query_api = self.client.query_api()
    
    query = f'from(bucket: "{self.bucket}")\
        |> range(start: -3d)\
        |> filter(fn: (r) => r.branch == "{branch}")\
        |> filter(fn: (r) => r["endpoint"] == "{self.sensor_type}")\
        |> aggregateWindow(every: 1m, fn: mean, createEmpty: false)\
        |> yield(name: "sensor_value")'
    
    print("Query: ",query)
    result_df = query_api.query_data_frame(query=query)
    return result_df

In [ ]:
# 전력량 데이터 가져오기
def query_energy(self, branch):
    query_api = self.client.query_api()

    query = f'from(bucket: "{self.bucket}")\
            |> range(start: -3d)\
            |> filter(fn: (r) => r["branch"] == "{branch}")\
            |> filter(fn: (r) => r["endpoint"] == "{self.sensor_type}")\
            |> filter(fn: (r) => r["phase"] == "total")\
            |> filter(fn: (r) => r["description"] == "w")\
            |> group(columns: ["site"])\
            |> aggregateWindow(every: 1m, fn: mean, createEmpty: false)\
            |> yield(name: "sensor_value")'
    
    print("Query: ", query)
    result_df = query_api.query_data_frame(query=query)
    return result_df